- model subclassing, can let you make classes for different blocks, so you dont have to retype code

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
class CNNBlock(layers.Layer):
    #this class is inheriting from layers.layers
    #below is the constructor of the class
    #self is just the instance of the class you are working on, so if you make an instance of a class, its pointer and self will have the same value

    #this function is basically initiliazing all the layers we are going to use
    def __init__(self, out_channels,kernel_size=3):
      #super helps with inheritance, allows one class to access the methods and attributes of another class in the same hierechy  +
      super(CNNBlock,self).__init__()
      self.conv = layers.Conv2D(out_channels,kernel_size,padding="same")
      self.bn = layers.BatchNormalization()

    #the call function is just placing the input tensor into these
    def call(self,input_tensor,training=False):
      x = self.conv(input_tensor)
      x = self.bn(x,training=training)
      x=tf.nn.relu(x)
      return x

model =keras.Sequential(
  [
    CNNBlock(32),
    CNNBlock(64),
    CNNBlock(128),
    layers.Flatten(),
    layers.Dense(10)
  ]
)

model.compile(
  optimizer=keras.optimizers.Adam(),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=["accuracy"]
)

model.fit(x_train,y_train,epochs=5)
model.evaluate(x_test,y_test)

Epoch 1/5
1875/1875 [==============================] - 23s 6ms/step - loss: 0.5366 - accuracy: 0.9514
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0496 - accuracy: 0.9852
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0431 - accuracy: 0.9865
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0348 - accuracy: 0.9893
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.0561 - accuracy: 0.9838


[0.05611175671219826, 0.9837999939918518]

In [ ]:
class ResBlock(layers.Layer):
  def __init__(self,channels):
    super(ResBlock,self).__init__()
    self.cnn1=CNNBlock(channels[0])
    self.cnn2=CNNBlock(channels[1])
    self.cnn3=CNNBlock(channels[2])
    self.pooling=layers.MaxPool2D()

    #similar to skip connections below ----------- make it a 1x1 convolution becasue this will generate the same size, 1x1 conv will reshape and keep dimensions
    self.identity_mapping=layers.Conv2D(channels[1],1,padding='same')

  def call(self,input_tensor,training=False):
    x=self.cnn1(input_tensor,training=training)
    x=self.cnn2(x,training=training)
    #doing the skip connections here, adding the input tensor before traveling through blocks and the one that travels through blocks
    x=self.cnn3(
      x + self.identity_mapping(input_tensor),training=training,

    )
    return self.pooling(x)

#you are inheriting from the keras.model class which has the functionality of layers.layer with extra stuff
class ResNet_Like(keras.Model):
  def __init__(self,num_classes=10):
    super(ResNet_Like,self).__init__()
    self.block1=ResBlock([32,32,64])
    self.block2=ResBlock([128,128,256])
    self.block3=ResBlock([128,256,512])
    self.pool=layers.GlobalAveragePooling2D()
    self.classifier=layers.Dense(num_classes)

  def call(self,input_tensor,training=False):
    x=self.block1(input_tensor,training=training)
    x=self.block2(x,training=training)
    x=self.block3(x,training=training)
    x=self.pool(x)
    return self.classifier(x)

model=ResNet_Like(num_classes=10)
model.compile(
  optimizer=keras.optimizers.Adam(),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=["accuracy"]
)

model.fit(x_train,y_train,batch_size=64,epochs=1,verbose=2)
print(model.summary())
model.evaluate(x_test,y_test,batch_size=64,verbose=2)

938/938 - 22s - loss: 0.0851 - accuracy: 0.9741 - 22s/epoch - 23ms/step
Model: "res_net__like"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_block (ResBlock)        multiple                  28640     
                                                                 
 res_block_1 (ResBlock)      multiple                  526976    
                                                                 
 res_block_2 (ResBlock)      multiple                  1839744   
                                                                 
 global_average_pooling2d (  multiple                  0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense_1 (Dense)             multiple                  5130      
                                                                 
Total params: 2400490 (9.16 MB)
Trainable param

[0.13928164541721344, 0.9545000195503235]